# Exploratory Data Analysis and Preprocessing for Moodle's Event Log

Moodle's Event Log is often used as dataset in research on Process Mining, but there are some problems that exist in the event log. Those problems are:
1. Time Column didn't use a proper time format
2. There's too many scenario which difficult to understand
3. Sometimes there's still anomaly in the event log

In this notebook we will do preprocessing and filtering on Moodle's event log on quiz taking to prepare it for Process Mining <br>

Moodle's event log consist of 9 columns
1. Time
2. User full name
3. Affected user
4. Event context
5. Component
6. Event name
7. Description
8. Origin
9. Ip address

The preprocessing and filtering will consist of 7 main phases, convert time format, giving alias and filtering, count quiz attempt, join columns, drop columns, choose 3 main columns, and applying simple heuristic filtering

![alt text](img/tahapan_jupyter.png "Phase")



### Importing library

In [311]:
import numpy as np, pandas as pd
from datetime import datetime

### Create Preprocessing class and SimpleHeuristic class

In [312]:
class Preprocessing:
    """Preprocessing Class"""
    
    def __init__(self, file):
        """Initialize class"""
        
        self.data = pd.read_csv(file)
        
    def getColumns(self):
        """Return list of columns in data"""
        
        return {'data': list(self.data.columns),'message': 'Succesfully fetch the data','status': 'success'}
    
    def getUniqueData(self):
        """Handle GET request to display data for giving alias"""
        
        # Initialize unique data array
        # Store unique data in every column
        unique_data = {}
        
        for col in self.data.columns:
            # Get all unique values in a column
            unique_data[col] = list(set(self.data[col]))
        
        # Return data to front end
        return {"data":{"column": list(self.data.columns),"data": unique_data}}
    
    def convertTime(self, column):
        """Convert time format to ISO format"""
        
        try:
            # Initialize array to store new formatted time
            new_time =  []
            
            for idx, time in enumerate(self.data[column]):
                new = datetime.strptime(time, '%d/%m/%y, %H:%M')
                new_time.append(new)
                
            # Replace time
            self.data[column] = new_time
            
            return 'Success'
        
        except Exception as e:
            return e
    
    def alias(self, alias, column):
        """Handle POST request to map the alias data"""
        
        # Map value on spesific column
        self.data[column] = self.data[column].map(alias)
        
        # Remove any missing value
        self.data[column].replace('', np.nan, inplace=True)
        self.data.dropna(subset=[column], inplace=True)
        
        return 'Success'
    
    def countAttempt(self, base, count, start, end):
        """Count quiz attempted by every student"""
        
        # Get unique case id
        id_list = self.data[base].unique()
        
        # Store data per case id
        data_per_id = {}
        for id in id_list:
            if id not in data_per_id:
                data_per_id[id] = self.data[self.data[base] == id]
                
        # Initialize new data
        list_of_data = []
        # Count every attempt
        for key, data in data_per_id.items():
            dt = data
            time_id_list = []
            id = np.nan
            iter = 0
            for x in range(len(dt.index)):
                if dt[count].iloc[x] == start:
                    iter+=1
                    id = str(iter)
                elif x != 0:
                    if dt[count].iloc[x-1] == end:
                        id = np.nan
                time_id_list.append(id)
            dt['n_attempt'] = time_id_list
            dt.dropna(subset=['n_attempt'], inplace=True)
            list_of_data.append(dt)
            
        # Concatenate all data
        self.data = pd.concat(list_of_data)
        # Sort the index
        self.data.sort_index(inplace=True)
        
        return 'Success'
    
    def joinColumn(self, column1, column2, columnName, delimiter):
        """Handle POST request to join two columns together"""
        
        if delimiter != None:
            self.data[columnName] = self.data[column1].map(str) + str(delimiter) + self.data[column2].map(str)
        else:
            self.data[columnName] = self.data[column1].map(str) + self.data[column2].map(str)
        
        return 'Success'
    
    def dropColumn(self, column):
        """Handle POST request to drop a column from data"""
        
        self.data.drop(column, axis=1, inplace=True)
        
        return 'Success'
    
    def chooseColumn(self, case_id, event, timestamp):
        """Choose 3 main columns (case id, event, and timestamp)"""
        
        try:
            # Arrange new input
            new_input = [case_id, event, timestamp]
            # Get columns from current data
            old_columns = list(self.data.columns)
            
            # Initialize array to store new column
            new_columns = []
            # Prepare renamed columns
            renamed_columns = ['case_id','event','timestamp']
            
            for val in new_input:
                new_columns.append(old_columns[val])
                
            for idx, val in enumerate(old_columns):
                for val2 in new_input:
                    if idx != val2 and val not in new_columns:
                        new_columns.append(val)
                        renamed_columns.append(val)
            
            # Re-arrange column
            self.data = self.data[new_columns]
            # Rename columns
            self.data.columns = renamed_columns
            
            return 'Success'
        
        except Exception as e:
            return e

In [313]:
class Statistic:
    """Summary Statistic Class"""
    
    def __init__(self, data):
        if 'case_id' in data.columns and 'event' in data.columns and 'timestamp' in data.columns:
            self.data = data
        else:
            raise Exception('Sorry your data is not ready yet')
        
    def summary(self):
        """Summary of the entire event log"""
        
        # Total number of case
        total_case = len(self.data['case_id'].unique())
        # Total number of event
        total_event = len(self.data['event'])
        # Total number of unique event
        jenis_event = len(self.data['event'].unique())

        # All unique event
        all_event = list(set(self.data['event']))
        # Calculate the occurance of every event
        all_event_occurance = []
        for event in all_event:
            event_occurance = {}
            occurance = len(self.data[self.data['event'] == event])
            percentage = occurance / total_event
            event_occurance['event'] = event
            event_occurance['absolute'] = occurance
            event_occurance['relative'] = percentage * 100
            all_event_occurance.append(event_occurance)

        # Sort all value
        all_event_occurance.sort(key=self.absoluteColumn, reverse=True)
        
        return {
            'total_case': total_case,
            'total_event': total_event,
            'jenis_event': jenis_event,
            'event_occurance': all_event_occurance
        }
    
    def start(self):
        """Occurance of start event"""
        
        id_list = list(set(self.data['case_id']))
        data_per_id = {}
        for id in id_list:
            if id not in data_per_id:
                data_per_id[id] = self.data[self.data['case_id'] == id]
        starts = []
        for key, dt in data_per_id.items():
            for x in range(len(dt.index)):
                if x == 0:
                    starts.append(dt['event'].iloc[x])
                elif dt['case_id'].iloc[x-1] != dt['case_id'].iloc[x]:
                    starts.append(dt['event'].iloc[x])
        start_list = list(set(starts))

        start_dict = {}
        total_in_start = 0

        for event in starts:
            if event not in start_dict:
                start_dict[event] = 1
                total_in_start+=1
            else:
                start_dict[event]+=1
                total_in_start+=1

        # Dictionary occurance start and end event
        start_occurance = []

        for key, item in start_dict.items():
            start = {}
            occurance = item
            percentage = occurance/total_in_start
            start['event'] = key
            start['absolute'] = occurance
            start['relative'] = percentage * 100
            start_occurance.append(start)

        # Sort all value
        start_occurance.sort(key=self.absoluteColumn, reverse=True)
        
        return {
            'start_event': start_list,
            'start_occurance': start_occurance
        }
    
    def end(self):
        # List of all the case id
        id_list = list(set(self.data['case_id']))
        # Get data per id
        data_per_id = {}
        for id in id_list:
            if id not in data_per_id:
                data_per_id[id] = self.data[self.data['case_id'] == id]
        # List start event and end event
        ends = []
        for key, dt in data_per_id.items():
            for x in range(len(dt.index)):
                if x == len(dt.index)-1:
                    ends.append(dt['event'].iloc[x])
                elif dt['case_id'].iloc[x+1] != dt['case_id'].iloc[x]:
                    ends.append(dt['event'].iloc[x])
        end_list = list(set(ends))

        # Initialize dictionary to store end event
        end_dict = {}
        # Initialize sum of end event
        total_in_end = 0

        for event in ends:
            if event not in end_dict:
                end_dict[event] = 1
                total_in_end+=1
            else:
                end_dict[event]+=1
                total_in_end+=1

        # Dictionary occurance start and end event
        end_occurance = []

        for key, item in end_dict.items():
            end = {}
            occurance = item
            percentage = occurance/total_in_end
            end['event'] = key
            end['absolute'] = occurance
            end['relative'] = percentage * 100
            end_occurance.append(end)

        # Sort all value
        end_occurance.sort(key=self.absoluteColumn, reverse=True)
        
        return {
            'end_event': end_list,
            'end_occurance': end_occurance
        }
        
    def absoluteColumn(self, value):
        return value['absolute']

In [314]:
class SimpleHeuristic:
    """Simple Heuristic Filtering Class"""
    
    def __init__(self, data):
        if 'case_id' in data.columns and 'event' in data.columns and 'timestamp' in data.columns:
            self.data = data
        else:
            raise Exception('Sorry your data is not ready yet')
    
    def start(self):
        # Kolom untuk sort data ulang
        self.data['Sort'] = list([x for x in range(0,len(self.data['case_id']))])

        # List dari semua id
        id_list = self.data['case_id'].unique()

        # Mengambil data per id
        data_per_id = dict()
        for id in id_list:
            if id not in data_per_id:
                data_per_id[id] = self.data[self.data['case_id'] == id]

        # List of selected start event
        list_of_data = []
        for key, dt in data_per_id.items():
            for ev in start:
                if dt['event'].iloc[0] == ev:
                    list_of_data.append(dt)

        self.data = pd.concat(list_of_data)
        self.data.sort_values('Sort', inplace=True)
        self.data.drop('Sort', axis=1, inplace=True)
        
        return 'Success'
    
    def end(self):
        # Kolom untuk sort data ulang
        self.data['Sort'] = list([x for x in range(0,len(self.data['case_id']))])

        # List dari semua id
        id_list = self.data['case_id'].unique()

        # Mengambil data per id
        data_per_id = dict()
        for id in id_list:
            if id not in data_per_id:
                data_per_id[id] = self.data[self.data['case_id'] == id]

        # List selected end event
        list_of_data = []
        for key, dt in data_per_id.items():
            for ev in end:
                if dt['event'].iloc[-1] == ev:
                    list_of_data.append(dt)

        self.data = pd.concat(list_of_data)
        self.data.sort_values('Sort', inplace=True)
        self.data.drop('Sort', axis=1, inplace=True)
        
        return 'Success'
    
    def all(self):
        # Kolom untuk sort data ulang
        self.data['Sort'] = list([x for x in range(0,len(self.data['case_id']))])

        list_data = []
        for idx, ev in enumerate(self.data['event']):
            for evt in all:
                if ev == evt:
                    list_data.append(self.data.iloc[idx])

        data = pd.DataFrame(list_data)
        data.sort_values('Sort', inplace=True)
        data.drop('Sort', axis=1, inplace=True)
    
        return 'Success'

## Preprocessing

In [315]:
preprocessing = Preprocessing('logs_DS_20181002-1531_all_events.csv')

### 1. Convert time format

In [316]:
# Original format
preprocessing.data['Time'].head()

0    2/10/18, 15:31
1    2/10/18, 15:31
2    2/10/18, 15:30
3    2/10/18, 15:30
4    2/10/18, 15:30
Name: Time, dtype: object

In [317]:
preprocessing.convertTime('Time')

'Success'

In [318]:
# New format
preprocessing.data['Time'].head()

0   2018-10-02 15:31:00
1   2018-10-02 15:31:00
2   2018-10-02 15:30:00
3   2018-10-02 15:30:00
4   2018-10-02 15:30:00
Name: Time, dtype: datetime64[ns]

### 2. Filtering on component

In this notebook, we will only use quiz component

In [319]:
# Original value in component
preprocessing.data['Component'].unique().tolist()

['Logs',
 'System',
 'Quiz',
 'Forum',
 'URL',
 'User report',
 'Feedback',
 'File',
 'Overview report',
 'Live logs',
 'Statistics',
 'Course participation',
 'Activity report',
 'User tours']

In [320]:
preprocessing.alias({'Quiz':'Quiz'}, 'Component')

'Success'

In [321]:
# New value in component
preprocessing.data['Component'].unique().tolist()

['Quiz']

### 3. Filtering on user full name

We will change the value of user full name to number to hide their identity

In [322]:
maps = {}
for idx, value in enumerate(preprocessing.data['User full name'].unique().tolist()):
    maps[value] = idx

In [323]:
preprocessing.alias(maps, 'User full name')

'Success'

### 4. Give alias on event name

We will only use Quiz attempt started, Course module viewed, Quiz attempt viewed, Quiz attempt submitted, Quiz attempt summary viewed, Quiz attempt abandoned, and Quiz attempt reviewed from Event name column

In [324]:
maps = {
    'Quiz attempt started' : 'qas',
    'Course module viewed' : 'cmv',
    'Quiz attempt viewed' : 'qav',
    'Quiz attempt submitted' : 'qasub',
    'Quiz attempt summary viewed' : 'qasv',
    'Quiz attempt abandoned' : 'qaban',
    'Quiz attempt reviewed ' : 'qar'
}

In [325]:
preprocessing.alias(maps, 'Event name')

'Success'

### 5. Give alias and filtering on event context

We will only use 1 quiz in this demo (Quiz: quiz 1 – pengenalan dunia digital)

In [326]:
preprocessing.alias({'Quiz: quiz 1 - pengenalan dunia digital':'q1'}, 'Event context')

'Success'

### 6. Count quiz attempt

We will count every quiz attempt that students do in Moodle<br>
1. Base column is student that attempt the quiz
2. Count column is column that represent the event
3. Start column is event that represent the start of quiz attempt
4. End column is event that represent the end of quiz attempt

In [327]:
base = 'User full name'
count = 'Event name'
start = 'qas'
end = 'qasub'

In [328]:
preprocessing.countAttempt(base, count, start, end)

c:\users\demas\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\demas\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'Success'

### 7. Join event name and event context

We will join event name and event context column so we can see which event corespond to which quiz

In [329]:
preprocessing.joinColumn('Event context', 'Event name', 'event', '_')

'Success'

### 8. Join user full name and n_attempt

We will join user full name with n_attempt and use it as case id

In [330]:
preprocessing.joinColumn('User full name', 'n_attempt', 'case_id', '_')

'Success'

### 9. Drop affected user and Ip address

Since we won't need affected user and ip address, we will drop the columns

In [331]:
preprocessing.dropColumn('Affected user')

'Success'

In [332]:
preprocessing.dropColumn('IP address')

'Success'

### 10. Choose 3 main columns

We will need 3 main columns, which is case id, event, and timestamp<br>
case id is in 9th position, event is in 9th position and Time is in 0th position

In [333]:
preprocessing.chooseColumn(9, 8, 0)

'Success'

## Statistic

In [334]:
statistic = Statistic(preprocessing.data)

### 1. Summary Statistic

In [335]:
statistic.summary()

{'total_case': 989,
 'total_event': 3411,
 'jenis_event': 6,
 'event_occurance': [{'event': 'q1_cmv',
   'absolute': 1519,
   'relative': 44.532395192025795},
  {'event': 'q1_qas', 'absolute': 989, 'relative': 28.994429785986515},
  {'event': 'q1_qasub', 'absolute': 820, 'relative': 24.039871005570216},
  {'event': 'q1_qav', 'absolute': 58, 'relative': 1.7003811199061858},
  {'event': 'q1_qaban', 'absolute': 18, 'relative': 0.5277044854881267},
  {'event': 'q1_qasv', 'absolute': 7, 'relative': 0.20521841102316035}]}

### 2. Start event

In [336]:
statistic.start()

{'start_event': ['q1_qas'],
 'start_occurance': [{'event': 'q1_qas', 'absolute': 989, 'relative': 100.0}]}

### 3. End event

In [337]:
statistic.end()

{'end_event': ['q1_cmv', 'q1_qasub', 'q1_qav'],
 'end_occurance': [{'event': 'q1_qasub',
   'absolute': 820,
   'relative': 82.91203235591507},
  {'event': 'q1_cmv', 'absolute': 151, 'relative': 15.267947421638016},
  {'event': 'q1_qav', 'absolute': 18, 'relative': 1.820020222446916}]}

## Simple Heuristic Filtering